In [1]:
# Wavefunction generation
from pyscf import mcscf, fci, lib 
from pyscf import gto, scf, tools
import api as pyq 
import importlib
import os
import h5py
import pdb

print(pyq.__file__)


/Users/ksu/Documents/GitHub/pyqmc/pyqmc/api.py


In [29]:
# H2 molecule
def run_scf(scf_checkfile):
    mol = gto.M(atom="Li 0. 0. 0.; Li 0. 0. 2.", basis=f'ccecpccpvdz', unit='bohr')
    mf = scf.RHF(mol)
    mf.chkfile = scf_checkfile
    dm = mf.init_guess_by_atom()
    mf.kernel(dm)


def run_casscf(scf_checkfile, ci_checkfile):
    cell, mf = pyq.recover_pyscf(scf_checkfile, cancel_outputs=False)
    mc = mcscf.CASSCF(mf, 2, 2)
    mc.chkfile = ci_checkfile
    mc.kernel()
    with h5py.File(mc.chkfile, "a") as f:
        print("Available output from CASSCF:", f["mcscf"].keys())
        f["mcscf/nelecas"] = list(mc.nelecas)
        f["mcscf/ci"] = mc.ci
    return mc


def run_casci(scf_checkfile, ci_checkfile):
    cell, mf = pyq.recover_pyscf(scf_checkfile, cancel_outputs=False)
    mc = mcscf.CASCI(mf, 2, 2)
    mc.kernel()
    print(mc.__dict__.keys())
    with h5py.File(ci_checkfile, "a") as f:
        f.create_group("ci")
        f["ci/ncas"] = mc.ncas
        f["ci/nelecas"] = list(mc.nelecas)
        f["ci/ci"] = mc.ci
        f["ci/mo_coeff"] = mc.mo_coeff
        print("Available output from CASCI:", f["ci"].keys())
    return mc


def make_wf_object(scf_checkfile, ci_checkfile):
    mol, mf, mc = pyq.recover_pyscf(scf_checkfile, ci_checkfile=ci_checkfile)
    wf, _ = pyq.generate_wf(mol, mf, mc=mc)
    return wf

for fname in ['scf.hdf5','ci.hdf5']:
    if os.path.isfile(fname):
        os.remove(fname)

scf_checkfile = "scf.hdf5"
ci_checkfile = "ci.hdf5"
run_scf(scf_checkfile)
# run_casci(scf_checkfile, ci_checkfile)

converged SCF energy = -7.59749376733129


In [30]:
# Jastrow optimization
for fname in ['sj.hdf5']:
    if os.path.isfile(fname):
        os.remove(fname)

# pyq.OPTIMIZE(scf_checkfile, "sj.hdf5", ci_checkfile=ci_checkfile, max_iterations=2, verbose=False)
pyq.OPTIMIZE(scf_checkfile, "sj.hdf5", max_iterations=2, verbose=False)

In [31]:
# Jastrow optimization results
df = pyq.read_opt("sj.hdf5")
df

,energy,iteration,error,fname
0,-8.998524,0,0.024825,sj.hdf5
1,-12.373757,1,0.076862,sj.hdf5


In [ ]:
import recipes, abdmc, wftools, bosonwf, determinant_tools, mc
modules = [recipes, abdmc, wftools, bosonwf, determinant_tools, mc] # Reloads recipes.py file after each update
for m in modules:
    importlib.reload(m)
# Jastrow optimization
for fname in ['dmc.hdf5']:
    if os.path.isfile(fname):
        os.remove(fname)
from recipes import DMC
DMC("scf.hdf5", "dmc.hdf5", load_parameters="sj.hdf5", accumulators = {"energy": True}, verbose=False, nsteps = 500, branchtime = 10, tstep= 0.02)
# ABDMC("scf.hdf5", "dmc.hdf5", load_parameters="sj.hdf5", ci_checkfile=ci_checkfile, accumulators = {"energy": True}, verbose=False, nsteps = 500, branchtime = 10, tstep= 0.02)

> /Users/ksu/Documents/GitHub/pyqmc/pyqmc/slater.py(194)recompute()
    192             import pdb
    193             pdb.set_trace()
--> 194             self._dets.append(
    195                 gpu.cp.asarray(np.linalg.slogdet(mo_vals))
    196             )  # Spin, (sign, val), nconf, [ndet_up, ndet_dn]



ipdb>  c


> /Users/ksu/Documents/GitHub/pyqmc/pyqmc/slater.py(193)recompute()
    191             mo_vals = gpu.cp.swapaxes(mo[:, :, self._det_occup[s]], 1, 2)
    192             import pdb
--> 193             pdb.set_trace()
    194             self._dets.append(
    195                 gpu.cp.asarray(np.linalg.slogdet(mo_vals))



ipdb>  mo_vals.shape


(1000, 1, 3, 3)


...

0


ipdb>  aa=s
ipdb>  aa


0


ipdb>  self._nelec


(3, 3)
